### This notebook investigates the simple question of how much of the Earth is raining using one day of IMERG data.

Assumption:
* rainfall is statistically constant over one day (true to first order), and
* IMERG (restricted to ±60° latitudes) is representative of global precipitation (pretty good).

Caveat: IMERG is primarily constructed from space-borne microwave instruments, which may encounter challenges picking up very light rain (< 0.1 mm / hr).

IMERG data can be downloaded here: http://pmm.nasa.gov/data-access/downloads/gpm.

In [1]:
import numpy as np
import h5py
from glob import glob

In [2]:
imergpath = '/media/Sentinel/data/IMERG/'
year, month, day = 2014, 4, 1

Read the IMERG files for one day.

In [3]:
inpath = '%s%4d/%02d/%02d/' % (imergpath, year, 4, 1)
files = sorted(glob('%s3B-HHR*' % inpath))
nt = len(files)

with h5py.File(files[0]) as f:
    lats = f['Grid/lat'][:]
    lons = f['Grid/lon'][:]
    fillvalue = f['Grid/precipitationCal'].attrs['_FillValue']
    
nlon, nlat = len(lons), len(lats)

In [4]:
Pimerg = np.ma.masked_all([nt, nlon, nlat], dtype = np.float32)

inpath = '%s%4d/%02d/%02d/' % (imergpath, year, month, day)
files = sorted(glob('%s3B-HHR*' % inpath))

for tt in range(nt):
    with h5py.File(files[tt]) as f:
        Pimerg[tt] = np.ma.masked_less(f['Grid/precipitationCal'][:], -100.)

Calculate the ratio of the number of grid boxes with rain (above 0.01 mm / h threshold) to the number of valid grid boxes (mainly between ±60° latitudes).

In [5]:
print(np.ma.sum(Pimerg > 0.01) / np.ma.count(Pimerg))

0.0551275945216


Therefore, about **5.5% of the Earth's area is raining at any one time**.

The simplification made here is that the ratio of *grid boxes* is similar to the ratio of *areas*. Or, putting it in another way, a grid box at high latitude has a smaller area than a grid box at the equator and thus contribute less to the ratio. However, to first order this is a fair assumption. Improvements can be made by simply weighting the grid boxes by the cosine of the latitude.

In [6]:
# print system info: please manually input non-core imported modules
%load_ext version_information
%version_information numpy, h5py

Software versions
Python 3.3.2 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-4)]
IPython 3.1.0
OS Linux 2.6.32 504.1.3.el6.x86_64 x86_64 with centos 6.6 Final
numpy 1.9.2
h5py 2.5.0
Fri May 29 08:07:55 2015 EDT